# PaperPi
![lms_client plugin](./paperpi/plugins/splash_screen/splash_screen_sample.png)

E-Paper display with multiple display plugins.

PaperPi is designed to work with SPI e-paper/e-ink displays with long refresh delays. It has been specifically written to work with the [WaveShare](https://www.waveshare.com/product/displays/e-paper.htm) SPI displays. 

PaperPi rotates through a user-configured selection of plugins each represented by a single static "screen." 


## Plugins
PaperPi supports many different plugins and layouts for each plugin. The plugin structure is open and documented to allow building your own plugins or customizing existing plugins.

[Complete Plugins  List](./documentation/Plugins.md)

| | | |
|:-------------------------:|:-------------------------:|:-------------------------:|
|<img src=./paperpi/plugins/librespot_client/librespot_client_sample.png alt="librespot plugin" width=200 />LibreSpot (spotify) Plugin|<img src=./paperpi/plugins/word_clock/word_clock_sample.png alt="word clock plugin" width=200 />Word Clock|<img src=./paperpi/plugins/lms_client/lms_client_sample.png alt="lms client plugin" width=200 />Logitech Media Server Plugin|
|<img src=./paperpi/plugins/dec_binary_clock/dec_binary_clock_sample.png alt="decimal binary clock" width=200 />Decimal-Binary Clock|<img src=./paperpi/plugins/met_no/met_no_sample.png alt="met_no plugin" width=200 />Met.no Weather|<img src=./paperpi/plugins/pi_dash/pi_dash_sample.png alt="Pi Dashboard" width=200 />Pi Dashboard|



## Requirements
* Raspberry Pi 4B
    - A Pi3 and possibly a Pi Zero are sufficient, but this are untested
* [WaveShare EPD SPI-only Screen](https://www.waveshare.com/product/displays/e-paper.htm)
    - UART, SPI/USB/I80 screens are not supported
    

    




In [ ]:
./paperpi/plugins/splash_screen/splash_screen_sample.png

In [1]:
!jupyter-nbconvert --to markdown README.ipynb

[NbConvertApp] Converting notebook README.ipynb to markdown
[NbConvertApp] Writing 0 bytes to README.md
